In [1]:
!pip install requests beautifulsoup4 pandas openpyxl

### Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

**Function to construct the search URL**

In [3]:
def get_search_url(site, search_term):
    """Generate search URL based on site and search term."""
    if site.lower() == 'amazon':
        return f"https://www.amazon.in/s?k={search_term.replace(' ', '+')}"
    elif site.lower() == 'flipkart':
        return f"https://www.flipkart.com/search?q={search_term.replace(' ', '%20')}"
    else:
        raise ValueError("Website not supported. Please use 'Amazon' or 'Flipkart'.")

**Function to extract product details**

In [4]:
def get_search_url(site, search_term):
    """Generate search URL based on site and search term."""
    if site.lower() == 'amazon':
        return f"https://www.amazon.in/s?k={search_term.replace(' ', '+')}"
    elif site.lower() == 'flipkart':
        return f"https://www.flipkart.com/search?q={search_term.replace(' ', '%20')}"
    else:
        raise ValueError("Website not supported. Please use 'Amazon' or 'Flipkart'.")

**Function to extract product details**

In [5]:
def extract_product_details(site, search_term):
    """Scrape product details from the given site and search term."""
    url = get_search_url(site, search_term)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    products = []

**Amazon scraping**

In [8]:
# Function to extract product details
def extract_product_details(site, search_term):
    """Scrape product details from the given site and search term."""
    url = get_search_url(site, search_term)
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    products = []
    
    # Amazon scraping
    if site.lower() == 'amazon':
        product_list = soup.find_all('div', {'data-component-type': 's-search-result'})
        for item in product_list:
            try:
                name = item.h2.text.strip()
            except AttributeError:
                name = "Name not available"
            
            try:
                price_whole = item.find('span', 'a-price-whole')
                price_fraction = item.find('span', 'a-price-fraction')
                if price_whole and price_fraction:
                    price = f"{price_whole.text.strip()}.{price_fraction.text.strip()}"
                else:
                    price = "Price not available"
            except AttributeError:
                price = "Price not available"
            
            try:
                rating = item.find('span', class_='a-icon-alt').text.strip()
            except AttributeError:
                rating = "Rating not available"
            
            products.append([name, price, rating])

    # Flipkart scraping (as previously defined)
    elif site.lower() == 'flipkart':
        product_list = soup.find_all('div', {'class': '_1AtVbE'})
        for item in product_list:
            name_tag = item.find('a', {'class': 'IRpwTa'})
            if not name_tag:
                continue
            name = name_tag.text.strip()
            try:
                price = item.find('div', {'class': '_30jeq3'}).text.strip()
            except AttributeError:
                price = "Price not available"
            try:
                rating = item.find('div', {'class': '_3LWZlK'}).text.strip()
            except AttributeError:
                rating = "Rating not available"
            products.append([name, price, rating])

    return products


**Function to save data to Excel**

In [10]:
def save_to_excel(products, filename):
    """Save the scraped product data to an Excel file."""
    df = pd.DataFrame(products, columns=['Product Name', 'Price', 'Rating'])
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"Data saved to {filename}")

**Main function to search and save product data**

In [13]:
def search_and_save(site, search_term, filename="product_details.xlsx"):
    """Main function to scrape product data and save to Excel."""
    products = extract_product_details(site, search_term)
    if products:
        save_to_excel(products, filename)
    else:
        print("No products found.")

**Usage**

In [ ]:
# Run this cell to perform the search and save results to an Excel file
site = input("Enter e-commerce site (Amazon or Flipkart): ").strip().lower()
search_term = input("Enter the product search term: ").strip()
search_and_save(site, search_term)